In [1]:
!ls ../../../models/

robertuito-lince-pos  robertuito-lince-sentiment  robertuito-lince-sentiment-2


In [57]:
%load_ext autoreload
%autoreload 2
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from pysentimiento.lince.sentiment import load_datasets

model_path = "../../../models/robertuito-lince-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(model_path)


tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.model_max_length = 128

_, _, test_dataset = load_datasets(lang="es")


loading configuration file ../../../models/robertuito-lince-sentiment/config.json
Model config RobertaConfig {
  "_name_or_path": "../../../models/robertuito-lince-sentiment",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0",
  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at ../../../models/robertuito-lince-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
Didn't find file ../../../models/robertuito-lince-sentiment/added_tokens.json. We won't load it.
loading file None
loading file ../../../models/robertuito-lince-sentiment/special_tokens_map.json
loading file ../../../models/robertuito-lince-sentiment/tokenizer_config.json
loading file ../../../models/robertuito-lince-sentiment/tokenizer.json
Reusing dataset lince (/root/.cache/huggingface/datasets/lince/sa_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/sa_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-029bff53c5a1fd47.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/sa_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-818d6ebd3452f42c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/sa_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-419ab4c207a77fce.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/sa_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-53d17b0587842a60.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/sa_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-eb8144edb12d19a4.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/

In [58]:

def tokenize(batch):
    return tokenizer(batch['text'], padding=False, truncation=True)

test_dataset = test_dataset.map(
    tokenize, batched=True, batch_size=32, 
)
test_dataset = test_dataset.remove_columns(["label"])

Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/sa_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-6876735a70d370be.arrow


In [59]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

train_args = TrainingArguments(
    output_dir="./test/",
    per_device_eval_batch_size=32,
    do_train=False,
)

trainer_args = {
    "model": model,
    "args": train_args,
    "eval_dataset": test_dataset,
    "tokenizer": tokenizer,
}

eval_trainer = Trainer(**trainer_args)
ret = eval_trainer.predict(test_dataset)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, text, lid, words, sa.
***** Running Prediction *****
  Num examples = 4736
  Batch size = 32


In [60]:
ret.predictions.shape

(4736, 3)

In [61]:
assert ret.predictions.shape[0] == len(test_dataset)

In [62]:
from tqdm.auto import tqdm
from pysentimiento.lince.sentiment import id2label

outputs = []

for pred in tqdm(ret.predictions):
    pred = id2label[pred.argmax()]
    outputs.append(pred)

  0%|          | 0/4736 [00:00<?, ?it/s]

In [63]:
from collections import Counter

Counter(outputs)

Counter({'neutral': 1115, 'positive': 2941, 'negative': 680})

In [64]:
submission_path = "../submissions/01_robertuito/sa_spa_eng.txt"

with open(submission_path, "w") as f:
    for i, output in enumerate(outputs):
        f.write(output + "\n")

In [62]:
import torch
sum(map(torch.numel, model.parameters()))

108212755